In [ ]:
!pip install lightgbm


In [ ]:
!pip install catboost

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.base import clone
import numpy as np
import pandas as pd

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.9 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()
import pandas as pd

train = pd.read_csv('track2_train_participant.csv')
test = pd.read_csv('track2_test_participant.csv')



Saving track2_test_participant.csv to track2_test_participant (1).csv
Saving track2_train_participant.csv to track2_train_participant (1).csv


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving track2_metadata.xlsx to track2_metadata.xlsx


In [ ]:
# prompt: I want to fill the most frequent label in '도유' Column

# Fill missing values in '도유' column with the most frequent label
most_frequent_label = train['도유'].mode()[0]
train['도유'].fillna(most_frequent_label, inplace=True)
test['도유'].fillna(most_frequent_label, inplace=True)

train['강종'] = train['강종'].fillna('일반')
test['강종'] = test['강종'].fillna('일반')

train['소재길이'] = train['소재길이'].fillna(train['소재길이'].mean())
test['소재길이'] = test['소재길이'].fillna(test['소재길이'].mean())

train['장력1'] = train['장력1'].fillna(train['장력1'].median())
test['장력1'] = test['장력1'].fillna(test['장력1'].median())

train['스피드1'] = train['스피드1'].fillna(train['스피드1'].median())
test['스피드1'] = test['스피드1'].fillna(test['스피드1'].median())

train['장력2'] = train['장력2'].fillna(train['장력2'].median())
test['장력2'] = test['장력2'].fillna(test['장력2'].median())

train['스피드2'] = train['스피드2'].fillna(train['스피드2'].median())
test['스피드2'] = test['스피드2'].fillna(test['스피드2'].median())

train['장력3'] = train['장력3'].fillna(train['장력3'].median())
test['장력3'] = test['장력3'].fillna(test['장력3'].median())

train['스피드3'] = train['스피드3'].fillna(train['스피드3'].median())
test['스피드3'] = test['스피드3'].fillna(test['스피드3'].median())

<ipython-input-175-c9f3bef460b5>:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train['도유'].fillna(most_frequent_label, inplace=True)
<ipython-input-175-c9f3bef460b5>:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', t

In [ ]:
# prompt: I want to count na in train

print(train.isna().sum())
print(test.isna().sum())


월          0
일자         0
두께         0
소재폭        0
제품폭        0
마진         0
두께그룹       0
폭그룹        0
마진그룹       0
강종         0
품명그룹       0
기준장력       0
No.        0
생산일        0
라인         0
생산지시번호     0
작업장        0
소재번호       0
소유         0
품명         0
유형         0
재질         0
Size       0
수량         0
소재중량       0
소재길이       0
제품번호       0
제품구분       0
생산중량       0
수율(%)      0
Size.1     0
단중         0
중량         0
Gross중량    0
도유         0
장력1        0
스피드1       0
장력2        0
스피드2       0
장력3        0
스피드3       0
장력         0
스피드        0
수요가        0
OK         0
NG         0
dtype: int64
월          0
일자         0
두께         0
소재폭        0
제품폭        0
마진         0
두께그룹       0
폭그룹        0
마진그룹       0
강종         0
품명그룹       0
기준장력       0
No.        0
생산일        0
라인         0
생산지시번호     0
작업장        0
소재번호       0
소유         0
품명         0
유형         0
재질         0
Size       0
수량         0
소재중량       0
소재길이       0
제품번호       0
제품구분       0
생산중량       0
수율(%)      0

In [ ]:
y = train['OK']
x = train.drop(columns = ['OK','NG', '라인', '작업장', '소유', '유형', '수량', '제품구분', '수요가'])

x_test = test.drop(columns = ['라인', '작업장', '소유', '유형', '수량', '제품구분', '수요가'])

'''
import re


def clean_range_expr_final(s):
    s = str(s)
    s = s.replace('≤', '<=').replace('＜', '<')  # 특수문자 통일
    s = s.replace(' ', '')                      # 전체 공백 제거
    s = s.replace('<=', ' <= ')                 # 올바른 위치에 공백 삽입
    s = s.replace('<', ' < ')
    s = re.sub(r'\s+', ' ', s).strip()          # 공백 정리
    return s


x['두께그룹'] = x['두께그룹'].apply(clean_range_expr_final)
x['폭그룹'] = x['폭그룹'].apply(clean_range_expr_final)
'''

x['두께그룹'] = x['두께그룹'].str.replace(' ', '')          # 모든 공백 제거
x['두께그룹'] = x['두께그룹'].str.replace('≤', '<=').str.replace('＜', '<')
x['두께그룹'] = x['두께그룹'].str.replace('<=', ' <= ').str.replace('<', ' < ')
x['두께그룹'] = x['두께그룹'].str.replace('≥', '>=').str.replace('>', '>')
x['두께그룹'] = x['두께그룹'].str.replace('>=', ' >= ').str.replace('>', ' > ')


x_test['두께그룹'] = x_test['두께그룹'].str.replace(' ', '')          # 모든 공백 제거
x_test['두께그룹'] = x_test['두께그룹'].str.replace('≤', '<=').str.replace('＜', '<')
x_test['두께그룹'] = x_test['두께그룹'].str.replace('<=', ' <= ').str.replace('<', ' < ')
x_test['두께그룹'] = x_test['두께그룹'].str.replace('≥', '>=').str.replace('>', '>')
x_test['두께그룹'] = x_test['두께그룹'].str.replace('>=', ' >= ').str.replace('>', ' > ')







# 폭그룹도 동일하게 처리
x['폭그룹'] = x['폭그룹'].str.replace(' ', '')
x['폭그룹'] = x['폭그룹'].str.replace('≤', '<=').str.replace('＜', '<')
x['폭그룹'] = x['폭그룹'].str.replace('<=', ' <= ').str.replace('<', ' < ')
x['폭그룹'] = x['폭그룹'].str.replace('≥', '>=').str.replace('>', '>')
x['폭그룹'] = x['폭그룹'].str.replace('>=', ' >= ').str.replace('>', ' > ')

x_test['폭그룹'] = x_test['폭그룹'].str.replace(' ', '')
x_test['폭그룹'] = x_test['폭그룹'].str.replace('≤', '<=').str.replace('＜', '<')
x_test['폭그룹'] = x_test['폭그룹'].str.replace('<=', ' <= ').str.replace('<', ' < ')
x_test['폭그룹'] = x_test['폭그룹'].str.replace('≥', '>=').str.replace('>', '>')
x_test['폭그룹'] = x_test['폭그룹'].str.replace('>=', ' >= ').str.replace('>', ' > ')




In [ ]:
x['newcol'] = x.apply(lambda row: get_ampere_from_matrix_v22(row['두께그룹'], row['폭그룹'], row['품명그룹']), axis=1)

x['newcol'].describe()

x_test['newcol'] = x_test.apply(lambda row: get_ampere_from_matrix_v22(row['두께그룹'], row['폭그룹'], row['품명그룹']), axis=1)


In [ ]:
def get_ampere_from_matrix_v22(a,b,c):
  if c == 'GI 외' :
    if a == '0.4  < = t < = 0.5' :
        if b == '800  < = w < 900' : return 200
        if b == '900  < = w < 1000': return 200
        if b == '1000  < = w < 1100': return 200
        if b == '1100  < = w < 1200': return 200
        if b == '1200  < = w < 1300': return 200
        if b == '1300  < = w < 1400': return 200
        if b == '1400  < = w < 1500': return 200
        if b == '1500  < = w < 1600': return 200
        if b == '1600  < = w < 1700': return 200
        if b == '1700  < = w < 1800': return 200
        if b == '1800  < = w < 1850': return 200
    if a == '0.5  < = t < = 0.6':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 200
        if b == '1000  < = w < 1100': return 200
        if b == '1100  < = w < 1200': return 200
        if b == '1200  < = w < 1300': return 200
        if b == '1300  < = w < 1400': return 200
        if b == '1400  < = w < 1500': return 200
        if b == '1500  < = w < 1600': return 200
        if b == '1600  < = w < 1700': return 210
        if b == '1700  < = w < 1800': return 220
        if b == '1800  < = w < 1850': return 230
    if a == '0.6  < = t < = 0.7':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 200
        if b == '1000  < = w < 1100': return 200
        if b == '1100  < = w < 1200': return 200
        if b == '1200  < = w < 1300': return 200
        if b == '1300  < = w < 1400': return 200
        if b == '1400  < = w < 1500': return 220
        if b == '1500  < = w < 1600': return 230
        if b == '1600  < = w < 1700': return 250
        if b == '1700  < = w < 1800': return 260
        if b == '1800  < = w < 1850': return 270
    if a == '0.7  < = t < = 0.8':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 200
        if b == '1000  < = w < 1100': return 200
        if b == '1100  < = w < 1200': return 200
        if b == '1200  < = w < 1300': return 220
        if b == '1300  < = w < 1400': return 230
        if b == '1400  < = w < 1500': return 250
        if b == '1500  < = w < 1600': return 260
        if b == '1600  < = w < 1700': return 290
        if b == '1700  < = w < 1800': return 300
        if b == '1800  < = w < 1850': return 310
    if a == '0.8  < = t < = 0.9':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 200
        if b == '1000  < = w < 1100': return 210
        if b == '1100  < = w < 1200': return 230
        if b == '1200  < = w < 1300': return 250
        if b == '1300  < = w < 1400': return 270
        if b == '1400  < = w < 1500': return 280
        if b == '1500  < = w < 1600': return 290
        if b == '1600  < = w < 1700': return 310
        if b == '1700  < = w < 1800': return 330
        if b == '1800  < = w < 1850': return 340
    if a == '0.9  < = t < = 1.0':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 210
        if b == '1000  < = w < 1100': return 230
        if b == '1100  < = w < 1200': return 250
        if b == '1200  < = w < 1300': return 270
        if b == '1300  < = w < 1400': return 290
        if b == '1400  < = w < 1500': return 310
        if b == '1500  < = w < 1600': return 320
        if b == '1600  < = w < 1700': return 340
        if b == '1700  < = w < 1800': return 360
        if b == '1800  < = w < 1850': return 370
    if a == '1.0  < = t < = 1.1':
        if b == '800  < = w < 900': return 210
        if b == '900  < = w < 1000': return 230
        if b == '1000  < = w < 1100': return 250
        if b == '1100  < = w < 1200': return 270
        if b == '1200  < = w < 1300': return 290
        if b == '1300  < = w < 1400': return 320
        if b == '1400  < = w < 1500': return 340
        if b == '1500  < = w < 1600': return 350
        if b == '1600  < = w < 1700': return 360
        if b == '1700  < = w < 1800': return 390
        if b == '1800  < = w < 1850': return 400
    if a == '1.1  < = t < = 1.2':
        if b == '800  < = w < 900': return 230
        if b == '900  < = w < 1000': return 240
        if b == '1000  < = w < 1100': return 280
        if b == '1100  < = w < 1200': return 290
        if b == '1200  < = w < 1300': return 310
        if b == '1300  < = w < 1400': return 330
        if b == '1400  < = w < 1500': return 350
        if b == '1500  < = w < 1600': return 360
        if b == '1600  < = w < 1700': return 390
        if b == '1700  < = w < 1800': return 410
        if b == '1800  < = w < 1850': return 420
    if a == '1.2  < = t < = 1.3':
        if b == '800  < = w < 900': return 250
        if b == '900  < = w < 1000': return 250
        if b == '1000  < = w < 1100': return 300
        if b == '1100  < = w < 1200': return 310
        if b == '1200  < = w < 1300': return 320
        if b == '1300  < = w < 1400': return 340
        if b == '1400  < = w < 1500': return 370
        if b == '1500  < = w < 1600': return 380
        if b == '1600  < = w < 1700': return 400
        if b == '1700  < = w < 1800': return 420
        if b == '1800  < = w < 1850': return 430
    if a == '1.3  < = t < = 1.4':
        if b == '800  < = w < 900': return 270
        if b == '900  < = w < 1000': return 270
        if b == '1000  < = w < 1100': return 310
        if b == '1100  < = w < 1200': return 320
        if b == '1200  < = w < 1300': return 330
        if b == '1300  < = w < 1400': return 350
        if b == '1400  < = w < 1500': return 380
        if b == '1500  < = w < 1600': return 390
        if b == '1600  < = w < 1700': return 410
        if b == '1700  < = w < 1800': return 430
        if b == '1800  < = w < 1850': return 430
    if a == '1.4  < = t < = 1.5':
        if b == '800  < = w < 900': return 280
        if b == '900  < = w < 1000': return 290
        if b == '1000  < = w < 1100': return 320
        if b == '1100  < = w < 1200': return 340
        if b == '1200  < = w < 1300': return 350
        if b == '1300  < = w < 1400': return 360
        if b == '1400  < = w < 1500': return 390
        if b == '1500  < = w < 1600': return 420
        if b == '1600  < = w < 1700': return 430
        if b == '1700  < = w < 1800': return 430
        if b == '1800  < = w < 1850': return 430
    if a == '1.5  < = t < = 1.6':
        if b == '800  < = w < 900': return 300
        if b == '900  < = w < 1000': return 310
        if b == '1000  < = w < 1100': return 330
        if b == '1100  < = w < 1200': return 350
        if b == '1200  < = w < 1300': return 360
        if b == '1300  < = w < 1400': return 390
        if b == '1400  < = w < 1500': return 410
        if b == '1500  < = w < 1600': return 430
        if b == '1600  < = w < 1700': return 430
        if b == '1700  < = w < 1800': return 430
        if b == '1800  < = w < 1850': return 430
    if a == '1.6  < = t < = 1.7':
        if b == '800  < = w < 900': return 310
        if b == '900  < = w < 1000': return 320
        if b == '1000  < = w < 1100': return 340
        if b == '1100  < = w < 1200': return 360
        if b == '1200  < = w < 1300': return 380
        if b == '1300  < = w < 1400': return 410
        if b == '1400  < = w < 1500': return 430
        if b == '1500  < = w < 1600': return 430
        if b == '1600  < = w < 1700': return 430
        if b == '1700  < = w < 1800': return 430
        if b == '1800  < = w < 1850': return 430
    if a == '1.7  < = t < = 1.8':
        if b == '800  < = w < 900': return 330
        if b == '900  < = w < 1000': return 340
        if b == '1000  < = w < 1100': return 350
        if b == '1100  < = w < 1200': return 370
        if b == '1200  < = w < 1300': return 400
        if b == '1300  < = w < 1400': return 430
        if b == '1400  < = w < 1500': return 430
        if b == '1500  < = w < 1600': return 430
        if b == '1600  < = w < 1700': return 430
        if b == '1700  < = w < 1800': return 430
        if b == '1800  < = w < 1850': return 430
    if a == '1.8  < = t < = 1.9':
        if b == '800  < = w < 900': return 340
        if b == '900  < = w < 1000': return 360
        if b == '1000  < = w < 1100': return 360
        if b == '1100  < = w < 1200': return 390
        if b == '1200  < = w < 1300': return 430
        if b == '1300  < = w < 1400': return 430
        if b == '1400  < = w < 1500': return 430
        if b == '1500  < = w < 1600': return 430
        if b == '1600  < = w < 1700': return 430
        if b == '1700  < = w < 1800': return 430
        if b == '1800  < = w < 1850': return 430
    if a == '1.9  < = t < = 2.0':
        if b == '800  < = w < 900': return 350
        if b == '900  < = w < 1000': return 380
        if b == '1000  < = w < 1100': return 380
        if b == '1100  < = w < 1200': return 410
        if b == '1200  < = w < 1300': return 430
        if b == '1300  < = w < 1400': return 430
        if b == '1400  < = w < 1500': return 430
        if b == '1500  < = w < 1600': return 430
        if b == '1600  < = w < 1700': return 430
        if b == '1700  < = w < 1800': return 430
        if b == '1800  < = w < 1850': return 430
    else :
      return None


  if c == 'GI 오일링' :
    if a == '0.60  < = t < 0.70':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 200
        if b == '1000  < = w < 1100': return 200
        if b == '1100  < = w < 1200': return 210
        if b == '1200  < = w < 1300': return 220
        if b == '1300  < = w < 1400': return 240
        if b == '1400  < = w < 1500': return 260
        if b == '1500  < = w < 1600': return 280
        if b == '1600  < = w < 1700': return 300
        if b == '1700  < = w < 1800': return 320
    if a == '0.70  < = t < 0.80':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 200
        if b == '1000  < = w < 1100': return 220
        if b == '1100  < = w < 1200': return 240
        if b == '1200  < = w < 1300': return 260
        if b == '1300  < = w < 1400': return 280
        if b == '1400  < = w < 1500': return 300
        if b == '1500  < = w < 1600': return 320
        if b == '1600  < = w < 1700': return 330
        if b == '1700  < = w < 1800': return 340
    if a == '0.80  < = t < 0.90':
        if b == '800  < = w < 900': return 200
        if b == '900  < = w < 1000': return 220
        if b == '1000  < = w < 1100': return 250
        if b == '1100  < = w < 1200': return 270
        if b == '1200  < = w < 1300': return 290
        if b == '1300  < = w < 1400': return 320
        if b == '1400  < = w < 1500': return 340
        if b == '1500  < = w < 1600': return 360
        if b == '1600  < = w < 1700': return 370
        if b == '1700  < = w < 1800': return 380
    if a == '0.90  < = t < 1.0':
        if b == '800  < = w < 900': return 220
        if b == '900  < = w < 1000': return 250
        if b == '1000  < = w < 1100': return 280
        if b == '1100  < = w < 1200': return 300
        if b == '1200  < = w < 1300': return 330
        if b == '1300  < = w < 1400': return 360
        if b == '1400  < = w < 1500': return 360
        if b == '1500  < = w < 1600': return 370
        if b == '1600  < = w < 1700': return 380
        if b == '1700  < = w < 1800': return 390
    if a == '1.0  < = t < 1.1':
        if b == '800  < = w < 900': return 250
        if b == '900  < = w < 1000': return 280
        if b == '1000  < = w < 1100': return 300
        if b == '1100  < = w < 1200': return 330
        if b == '1200  < = w < 1300': return 360
        if b == '1300  < = w < 1400': return 370
        if b == '1400  < = w < 1500': return 370
        if b == '1500  < = w < 1600': return 380
        if b == '1600  < = w < 1700': return 400
        if b == '1700  < = w < 1800': return 430
    if a == '1.1  < = t < 1.2':
        if b == '800  < = w < 900': return 270
        if b == '900  < = w < 1000': return 300
        if b == '1000  < = w < 1100': return 330
        if b == '1100  < = w < 1200': return 370
        if b == '1200  < = w < 1300': return 370
        if b == '1300  < = w < 1400': return 390
        if b == '1400  < = w < 1500': return 390
        if b == '1500  < = w < 1600': return 410
        if b == '1600  < = w < 1700': return 440
    if a == '1.2  < = t < 1.3':
        if b == '800  < = w < 900': return 290
        if b == '900  < = w < 1000': return 330
        if b == '1000  < = w < 1100': return 340
        if b == '1100  < = w < 1200': return 370
        if b == '1200  < = w < 1300': return 380
        if b == '1300  < = w < 1400': return 390
        if b == '1400  < = w < 1500': return 420
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450
    if a == '1.3  < = t < 1.4':
        if b == '800  < = w < 900': return 320
        if b == '900  < = w < 1000': return 350
        if b == '1000  < = w < 1100': return 360
        if b == '1100  < = w < 1200': return 390
        if b == '1200  < = w < 1300': return 390
        if b == '1300  < = w < 1400': return 420
        if b == '1400  < = w < 1500': return 450
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450
    if a == '1.4  < = t < 1.5':
        if b == '800  < = w < 900': return 340
        if b == '900  < = w < 1000': return 360
        if b == '1000  < = w < 1100': return 370
        if b == '1100  < = w < 1200': return 400
        if b == '1200  < = w < 1300': return 420
        if b == '1300  < = w < 1400': return 450
        if b == '1400  < = w < 1500': return 450
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450
    if a == '1.5  < = t < 1.6':
        if b == '800  < = w < 900': return 370
        if b == '900  < = w < 1000': return 370
        if b == '1000  < = w < 1100': return 380
        if b == '1100  < = w < 1200': return 410
        if b == '1200  < = w < 1300': return 450
        if b == '1300  < = w < 1400': return 450
        if b == '1400  < = w < 1500': return 450
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450
    if a == '1.6  <= t < 1.7':
        if b == '800  < = w < 900' : return 370
        if b == '900  < = w < 1000': return 380
        if b == '1000  < = w < 1100': return 400
        if b == '1100  < = w < 1200': return 440
        if b == '1200  < = w < 1300': return 450
        if b == '1300  < = w < 1400': return 450
        if b == '1400  < = w < 1500': return 450
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450
    if a == '1.7  < = t < 1.8':
        if b == '800  < = w < 900': return 370
        if b == '900  < = w < 1000': return 390
        if b == '1000  < = w < 1100': return 420
        if b == '1100  < = w < 1200': return 450
        if b == '1200  < = w < 1300': return 450
        if b == '1300  < = w < 1400': return 450
        if b == '1400  < = w < 1500': return 450
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450
    if a == '1.8  < = t < 1.9':
        if b == '800  < = w < 900': return 380
        if b == '900  < = w < 1000': return 410
        if b == '1000  < = w < 1100': return 450
        if b == '1100  < = w < 1200': return 450
        if b == '1200  < = w < 1300': return 450
        if b == '1300  < = w < 1400': return 450
        if b == '1400  < = w < 1500': return 450
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450
    if a == 't  > = 1.9' :
        if b == '800  < = w < 900': return 390
        if b == '900  < = w < 1000': return 430
        if b == '1000  < = w < 1100': return 450
        if b == '1100  < = w < 1200': return 450
        if b == '1200  < = w < 1300': return 450
        if b == '1300  < = w < 1400': return 450
        if b == '1400  < = w < 1500': return 450
        if b == '1500  < = w < 1600': return 450
        if b == '1600  < = w < 1700': return 450

    else :
      return None


In [ ]:
'''
def get_ampere_from_matrix_v2(a,b,c):
  if c == 'GI 외' :
    if a == '0.4 < = t < = 0.5' :
        if b == '800 < = w < 900' : return 200
        if b == '900 < = w < 1000': return 200
        if b == '1000 < = w < 1100': return 200
        if b == '1100 < = w < 1200': return 200
        if b == '1200 < = w < 1300': return 200
        if b == '1300 < = w < 1400': return 200
        if b == '1400 < = w < 1500': return 200
        if b == '1500 < = w < 1600': return 200
        if b == '1600 < = w < 1700': return 200
        if b == '1700 < = w < 1800': return 200
        if b == '1800 < = w < 1850': return 200
    if a == '0.5 < = t < = 0.6':
        if b == '800 < = w < 900': return 200
        if b == '900 < = w < 1000': return 200
        if b == '1000 < = w < 1100': return 200
        if b == '1100 < = w < 1200': return 200
        if b == '1200 < = w < 1300': return 200
        if b == '1300 < = w < 1400': return 200
        if b == '1400 < = w < 1500': return 200
        if b == '1500 < = w < 1600': return 200
        if b == '1600 < = w < 1700': return 210
        if b == '1700 < = w < 1800': return 220
        if b == '1800 < = w < 1850': return 230
    if a == '0.6 <= t <= 0.7':
        if b == '800 <= w < 900': return 200
        if b == '900 <= w < 1000': return 200
        if b == '1000 <= w < 1100': return 200
        if b == '1100 <= w < 1200': return 200
        if b == '1200 <= w < 1300': return 200
        if b == '1300 <= w < 1400': return 200
        if b == '1400 <= w < 1500': return 220
        if b == '1500 <= w < 1600': return 230
        if b == '1600 <= w < 1700': return 250
        if b == '1700 <= w < 1800': return 260
        if b == '1800 <= w < 1850': return 270
    if a == '0.7 <= t <= 0.8':
        if b == '800 <= w < 900': return 200
        if b == '900 <= w < 1000': return 200
        if b == '1000 <= w < 1100': return 200
        if b == '1100 <= w < 1200': return 200
        if b == '1200 <= w < 1300': return 220
        if b == '1300 <= w < 1400': return 230
        if b == '1400 <= w < 1500': return 250
        if b == '1500 <= w < 1600': return 260
        if b == '1600 <= w < 1700': return 290
        if b == '1700 <= w < 1800': return 300
        if b == '1800 <= w < 1850': return 310
    if a == '0.8 <= t <= 0.9':
        if b == '800 <= w < 900': return 200
        if b == '900 <= w < 1000': return 200
        if b == '1000 <= w < 1100': return 210
        if b == '1100 <= w < 1200': return 230
        if b == '1200 <= w < 1300': return 250
        if b == '1300 <= w < 1400': return 270
        if b == '1400 <= w < 1500': return 280
        if b == '1500 <= w < 1600': return 290
        if b == '1600 <= w < 1700': return 310
        if b == '1700 <= w < 1800': return 330
        if b == '1800 <= w < 1850': return 340
    if a == '0.9 <= t <= 1.0':
        if b == '800 <= w < 900': return 200
        if b == '900 <= w < 1000': return 210
        if b == '1000 <= w < 1100': return 230
        if b == '1100 <= w < 1200': return 250
        if b == '1200 <= w < 1300': return 270
        if b == '1300 <= w < 1400': return 290
        if b == '1400 <= w < 1500': return 310
        if b == '1500 <= w < 1600': return 320
        if b == '1600 <= w < 1700': return 340
        if b == '1700 <= w < 1800': return 360
        if b == '1800 <= w < 1850': return 370
    if a == '1.0 <= t <= 1.1':
        if b == '800 <= w < 900': return 210
        if b == '900 <= w < 1000': return 230
        if b == '1000 <= w < 1100': return 250
        if b == '1100 <= w < 1200': return 270
        if b == '1200 <= w < 1300': return 290
        if b == '1300 <= w < 1400': return 320
        if b == '1400 <= w < 1500': return 340
        if b == '1500 <= w < 1600': return 350
        if b == '1600 <= w < 1700': return 360
        if b == '1700 <= w < 1800': return 390
        if b == '1800 <= w < 1850': return 400
    if a == '1.1 <= t <= 1.2':
        if b == '800 <= w < 900': return 230
        if b == '900 <= w < 1000': return 240
        if b == '1000 <= w < 1100': return 280
        if b == '1100 <= w < 1200': return 290
        if b == '1200 <= w < 1300': return 310
        if b == '1300 <= w < 1400': return 330
        if b == '1400 <= w < 1500': return 350
        if b == '1500 <= w < 1600': return 360
        if b == '1600 <= w < 1700': return 390
        if b == '1700 <= w < 1800': return 410
        if b == '1800 <= w < 1850': return 420
    if a == '1.2 <= t <= 1.3':
        if b == '800 <= w < 900': return 250
        if b == '900 <= w < 1000': return 250
        if b == '1000 <= w < 1100': return 300
        if b == '1100 <= w < 1200': return 310
        if b == '1200 <= w < 1300': return 320
        if b == '1300 <= w < 1400': return 340
        if b == '1400 <= w < 1500': return 370
        if b == '1500 <= w < 1600': return 380
        if b == '1600 <= w < 1700': return 400
        if b == '1700 <= w < 1800': return 420
        if b == '1800 <= w < 1850': return 430
    if a == '1.3 <= t <= 1.4':
        if b == '800 <= w < 900': return 270
        if b == '900 <= w < 1000': return 270
        if b == '1000 <= w < 1100': return 310
        if b == '1100 <= w < 1200': return 320
        if b == '1200 <= w < 1300': return 330
        if b == '1300 <= w < 1400': return 350
        if b == '1400 <= w < 1500': return 380
        if b == '1500 <= w < 1600': return 390
        if b == '1600 <= w < 1700': return 410
        if b == '1700 <= w < 1800': return 430
        if b == '1800 <= w < 1850': return 430
    if a == '1.4 <= t <= 1.5':
        if b == '800 <= w < 900': return 280
        if b == '900 <= w < 1000': return 290
        if b == '1000 <= w < 1100': return 320
        if b == '1100 <= w < 1200': return 340
        if b == '1200 <= w < 1300': return 350
        if b == '1300 <= w < 1400': return 360
        if b == '1400 <= w < 1500': return 390
        if b == '1500 <= w < 1600': return 420
        if b == '1600 <= w < 1700': return 430
        if b == '1700 <= w < 1800': return 430
        if b == '1800 <= w < 1850': return 430
    if a == '1.5 <= t <= 1.6':
        if b == '800 <= w < 900': return 300
        if b == '900 <= w < 1000': return 310
        if b == '1000 <= w < 1100': return 330
        if b == '1100 <= w < 1200': return 350
        if b == '1200 <= w < 1300': return 360
        if b == '1300 <= w < 1400': return 390
        if b == '1400 <= w < 1500': return 410
        if b == '1500 <= w < 1600': return 430
        if b == '1600 <= w < 1700': return 430
        if b == '1700 <= w < 1800': return 430
        if b == '1800 <= w < 1850': return 430
    if a == '1.6 <= t <= 1.7':
        if b == '800 <= w < 900': return 310
        if b == '900 <= w < 1000': return 320
        if b == '1000 <= w < 1100': return 340
        if b == '1100 <= w < 1200': return 360
        if b == '1200 <= w < 1300': return 380
        if b == '1300 <= w < 1400': return 410
        if b == '1400 <= w < 1500': return 430
        if b == '1500 <= w < 1600': return 430
        if b == '1600 <= w < 1700': return 430
        if b == '1700 <= w < 1800': return 430
        if b == '1800 <= w < 1850': return 430
    if a == '1.7 <= t <= 1.8':
        if b == '800 <= w < 900': return 330
        if b == '900 <= w < 1000': return 340
        if b == '1000 <= w < 1100': return 350
        if b == '1100 <= w < 1200': return 370
        if b == '1200 <= w < 1300': return 400
        if b == '1300 <= w < 1400': return 430
        if b == '1400 <= w < 1500': return 430
        if b == '1500 <= w < 1600': return 430
        if b == '1600 <= w < 1700': return 430
        if b == '1700 <= w < 1800': return 430
        if b == '1800 <= w < 1850': return 430
    if a == '1.8 <= t <= 1.9':
        if b == '800 <= w < 900': return 340
        if b == '900 <= w < 1000': return 360
        if b == '1000 <= w < 1100': return 360
        if b == '1100 <= w < 1200': return 390
        if b == '1200 <= w < 1300': return 430
        if b == '1300 <= w < 1400': return 430
        if b == '1400 <= w < 1500': return 430
        if b == '1500 <= w < 1600': return 430
        if b == '1600 <= w < 1700': return 430
        if b == '1700 <= w < 1800': return 430
        if b == '1800 <= w < 1850': return 430
    if a == '1.9 <= t <= 2.0':
        if b == '800 <= w < 900': return 350
        if b == '900 <= w < 1000': return 380
        if b == '1000 <= w < 1100': return 380
        if b == '1100 <= w < 1200': return 410
        if b == '1200 <= w < 1300': return 430
        if b == '1300 <= w < 1400': return 430
        if b == '1400 <= w < 1500': return 430
        if b == '1500 <= w < 1600': return 430
        if b == '1600 <= w < 1700': return 430
        if b == '1700 <= w < 1800': return 430
        if b == '1800 <= w < 1850': return 430
    return None

  if c == 'GI 오일링' :
    if a == '0.60 <= t < 0.70':
        if b == '800 <= w < 900': return 200
        if b == '900 <= w < 1000': return 200
        if b == '1000 <= w < 1100': return 200
        if b == '1100 <= w < 1200': return 210
        if b == '1200 <= w < 1300': return 220
        if b == '1300 <= w < 1400': return 240
        if b == '1400 <= w < 1500': return 260
        if b == '1500 <= w < 1600': return 280
        if b == '1600 <= w < 1700': return 300
        if b == '1700 <= w < 1800': return 320
    if a == '0.70 <= t < 0.80':
        if b == '800 <= w < 900': return 200
        if b == '900 <= w < 1000': return 200
        if b == '1000 <= w < 1100': return 220
        if b == '1100 <= w < 1200': return 240
        if b == '1200 <= w < 1300': return 260
        if b == '1300 <= w < 1400': return 280
        if b == '1400 <= w < 1500': return 300
        if b == '1500 <= w < 1600': return 320
        if b == '1600 <= w < 1700': return 330
        if b == '1700 <= w < 1800': return 340
    if a == '0.80 <= t < 0.90':
        if b == '800 <= w < 900': return 200
        if b == '900 <= w < 1000': return 220
        if b == '1000 <= w < 1100': return 250
        if b == '1100 <= w < 1200': return 270
        if b == '1200 <= w < 1300': return 290
        if b == '1300 <= w < 1400': return 320
        if b == '1400 <= w < 1500': return 340
        if b == '1500 <= w < 1600': return 360
        if b == '1600 <= w < 1700': return 370
        if b == '1700 <= w < 1800': return 380
    if a == '0.90 <= t < 1.0':
        if b == '800 <= w < 900': return 220
        if b == '900 <= w < 1000': return 250
        if b == '1000 <= w < 1100': return 280
        if b == '1100 <= w < 1200': return 300
        if b == '1200 <= w < 1300': return 330
        if b == '1300 <= w < 1400': return 360
        if b == '1400 <= w < 1500': return 360
        if b == '1500 <= w < 1600': return 370
        if b == '1600 <= w < 1700': return 380
        if b == '1700 <= w < 1800': return 390
    if a == '1.0 <= t < 1.1':
        if b == '800 <= w < 900': return 250
        if b == '900 <= w < 1000': return 280
        if b == '1000 <= w < 1100': return 300
        if b == '1100 <= w < 1200': return 330
        if b == '1200 <= w < 1300': return 360
        if b == '1300 <= w < 1400': return 370
        if b == '1400 <= w < 1500': return 370
        if b == '1500 <= w < 1600': return 380
        if b == '1600 <= w < 1700': return 400
        if b == '1700 <= w < 1800': return 430
    if a == '1.1 <= t < 1.2':
        if b == '800 <= w < 900': return 270
        if b == '900 <= w < 1000': return 300
        if b == '1000 <= w < 1100': return 330
        if b == '1100 <= w < 1200': return 370
        if b == '1200 <= w < 1300': return 370
        if b == '1300 <= w < 1400': return 390
        if b == '1400 <= w < 1500': return 390
        if b == '1500 <= w < 1600': return 410
        if b == '1600 <= w < 1700': return 440
    if a == '1.2 <= t < 1.3':
        if b == '800 <= w < 900': return 290
        if b == '900 <= w < 1000': return 330
        if b == '1000 <= w < 1100': return 340
        if b == '1100 <= w < 1200': return 370
        if b == '1200 <= w < 1300': return 380
        if b == '1300 <= w < 1400': return 390
        if b == '1400 <= w < 1500': return 420
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450
    if a == '1.3 <= t < 1.4':
        if b == '800 <= w < 900': return 320
        if b == '900 <= w < 1000': return 350
        if b == '1000 <= w < 1100': return 360
        if b == '1100 <= w < 1200': return 390
        if b == '1200 <= w < 1300': return 390
        if b == '1300 <= w < 1400': return 420
        if b == '1400 <= w < 1500': return 450
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450
    if a == '1.4 <= t < 1.5':
        if b == '800 <= w < 900': return 340
        if b == '900 <= w < 1000': return 360
        if b == '1000 <= w < 1100': return 370
        if b == '1100 <= w < 1200': return 400
        if b == '1200 <= w < 1300': return 420
        if b == '1300 <= w < 1400': return 450
        if b == '1400 <= w < 1500': return 450
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450
    if a == '1.5 <= t < 1.6':
        if b == '800 <= w < 900': return 370
        if b == '900 <= w < 1000': return 370
        if b == '1000 <= w < 1100': return 380
        if b == '1100 <= w < 1200': return 410
        if b == '1200 <= w < 1300': return 450
        if b == '1300 <= w < 1400': return 450
        if b == '1400 <= w < 1500': return 450
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450
    if a == '1.6 <= t < 1.7':
        if b == '800 <= w < 900' : return 370
        if b == '900 <= w < 1000': return 380
        if b == '1000 <= w < 1100': return 400
        if b == '1100 <= w < 1200': return 440
        if b == '1200 <= w < 1300': return 450
        if b == '1300 <= w < 1400': return 450
        if b == '1400 <= w < 1500': return 450
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450
    if a == '1.7 <= t < 1.8':
        if b == '800 <= w < 900': return 370
        if b == '900 <= w < 1000': return 390
        if b == '1000 <= w < 1100': return 420
        if b == '1100 <= w < 1200': return 450
        if b == '1200 <= w < 1300': return 450
        if b == '1300 <= w < 1400': return 450
        if b == '1400 <= w < 1500': return 450
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450
    if a == '1.8 <= t < 1.9':
        if b == '800 <= w < 900': return 380
        if b == '900 <= w < 1000': return 410
        if b == '1000 <= w < 1100': return 450
        if b == '1100 <= w < 1200': return 450
        if b == '1200 <= w < 1300': return 450
        if b == '1300 <= w < 1400': return 450
        if b == '1400 <= w < 1500': return 450
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450
    if a == 't >= 1.9' :
        if b == '800 <= w < 900': return 390
        if b == '900 <= w < 1000': return 430
        if b == '1000 <= w < 1100': return 450
        if b == '1100 <= w < 1200': return 450
        if b == '1200 <= w < 1300': return 450
        if b == '1300 <= w < 1400': return 450
        if b == '1400 <= w < 1500': return 450
        if b == '1500 <= w < 1600': return 450
        if b == '1600 <= w < 1700': return 450

    return None
'''

In [ ]:
from lightgbm import LGBMClassifier

for col in x.select_dtypes(include='object').columns:
    x[col] = x[col].astype('category')
model = LGBMClassifier(random_state = 42)
model.fit(x,y)

[LightGBM] [Info] Number of positive: 721, number of negative: 1119
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000558 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5074
[LightGBM] [Info] Number of data points in the train set: 1840, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391848 -> initscore=-0.439552
[LightGBM] [Info] Start training from score -0.439552
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


LGBMClassifier(random_state=42)

In [ ]:
from sklearn.metrics import classification_report, accuracy_score, f1_score
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

accuracy_list = []
f1_list = []

for fold, (train_idx, val_idx) in enumerate(kf.split(x, y)):
    print(f"\n📂 Fold {fold + 1} -----------------------")

    X_train, X_val = x.iloc[train_idx], x.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = LGBMClassifier(random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_val)

    acc = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    accuracy_list.append(acc)
    f1_list.append(f1)

    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score: {f1:.4f}")

# 평균 성능 출력
print("\n✅ 5-Fold 결과 요약")
print(f"평균 Accuracy: {np.mean(accuracy_list):.4f}")
print(f"평균 F1-score: {np.mean(f1_list):.4f}")


📂 Fold 1 -----------------------
[LightGBM] [Info] Number of positive: 577, number of negative: 895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000246 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4725
[LightGBM] [Info] Number of data points in the train set: 1472, number of used features: 38
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391984 -> initscore=-0.438981
[LightGBM] [Info] Start training from score -0.438981
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Accuracy: 0.9946
F1-score: 0.9931

📂 Fold 2 -----------------------
[LightGBM] [Info] Number of positive: 577, number of negative: 895
[LightGBM] [Info] Auto-choosing row-wise multi-thread

In [ ]:
# Convert categorical columns to numerical using one-hot encoding
x = pd.get_dummies(x, columns=x.select_dtypes(include=['object', 'category']).columns)
x_test = pd.get_dummies(x_test, columns=x_test.select_dtypes(include=['object', 'category']).columns)

# 모델 리스트
models = {"LightGBM": LGBMClassifier()}

# 결과 저장
results = []

# 교차검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 각 모델 평가
for name, model in models.items():
    f1_scores = []
    roc_auc_scores = []

    for train_idx, test_idx in cv.split(x, y):
        X_train, X_test = x.iloc[train_idx], x.iloc[test_idx]
        y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

        # CatBoost & NB는 스케일링 안함
        if name in ["Naive Bayes", "CatBoost"]:
            fitted_model = clone(model).fit(X_train, y_train)
        else:
            pipeline = Pipeline([
                ('scaler', StandardScaler()),
                ('classifier', clone(model))
            ])
            fitted_model = pipeline.fit(X_train, y_train)

        y_pred = fitted_model.predict(X_test)
        y_proba = fitted_model.predict_proba(X_test)[:, 1]  # ROC AUC용 확률 예측

        f1 = f1_score(y_test, y_pred)
        roc_auc = roc_auc_score(y_test, y_proba)

        f1_scores.append(f1)
        roc_auc_scores.append(roc_auc)

    # 가중합 계산
    f1_mean = np.mean(f1_scores)
    roc_auc_mean = np.mean(roc_auc_scores)
    weighted_score = 0.4 * f1_mean + 0.6 * roc_auc_mean

    results.append({
        "Model": name,
        "F1 Score": f1_mean,
        "ROC AUC": roc_auc_mean,
        "Weighted Score (0.4F1 + 0.6AUC)": weighted_score
    })

# 정렬 및 출력
df_results = pd.DataFrame(results).sort_values(by="Weighted Score (0.4F1 + 0.6AUC)", ascending=False)
print(df_results)

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 577, number of negative: 895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4496
[LightGBM] [Info] Number of data points in the train set: 1472, number of used features: 95
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391984 -> initscore=-0.438981
[LightGBM] [Info] Start training from score -0.438981
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 577, number of negative: 895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001640 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4497
[LightGBM] [Info] Number of data points in the train set: 1472, number of used features: 94
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391984 -> initscore=-0.438981
[LightGBM] [Info] Start training from score -0.438981
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


...

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 577, number of negative: 895
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001302 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4479
[LightGBM] [Info] Number of data points in the train set: 1472, number of used features: 90
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391984 -> initscore=-0.438981
[LightGBM] [Info] Start training from score -0.438981
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Info] Number of positive: 576, number of negative: 896
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001148 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4509
[LightGBM] [Info] Number of data points in the train set: 1472, number of used features: 95
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.391304 -> initscore=-0.441833
[LightGBM] [Info] Start training from score -0.441833
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
      Model  F1 Score   ROC AUC  Weighted Score (0.4F1 + 0.6AUC)
0  LightGBM  0.981962  0.998977                         0.992171


/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [ ]:
# Align the columns of x_test with those of x after one-hot encoding

# Get missing columns in x_test
missing_cols = set(x.columns) - set(x_test.columns)

# Add missing columns to x_test and fill with 0
for col in missing_cols:
    x_test[col] = 0

# Ensure the order of columns in x_test is the same as in x
x_test = x_test[x.columns]

# Now you can predict using the fitted model
y_pred_test = fitted_model.predict(x_test)
y_pred_test


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-188-225beb6ab8ec>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_test[col] = 0
<ipython-input-188-225beb6ab8ec>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  x_test[col] = 0
<ipython-input-188-225beb6ab8ec>:8: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = fra

array([0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1,
       1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,

In [ ]:

submission = pd.DataFrame({
    'id': range(len(y_pred_test)),
    'OK': (y_pred_test == 1).astype(int),
    'NG': (y_pred_test == 0).astype(int)
})

submission.to_csv("submission.csv", index=False)
from google.colab import files
files.download('submission.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>